In [201]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [202]:
path='/content/gdrive/MyDrive/Datasets/'
movies_df=pd.read_csv(path+'movies.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
ratings_df= pd.read_csv(path'ratings.csv',usecols=['userId', 'movieId', 'rating'],
            dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [203]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [204]:
ratings_df.head()


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [205]:
df = pd.merge(ratings_df,movies_df,on='movieId')
df.head()


,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [206]:
df= df.dropna(axis=0,subset=['title'])

In [207]:
ratings=pd.DataFrame(df.groupby('title')['rating'].mean())
ratings.rename(columns={'rating':'no_of_ratings'},inplace=True)
ratings.head(10)

,no_of_ratings
title,
'71 (2014),4.000000
'Hellboy': The Seeds of Creation (2004),4.000000
'Round Midnight (1986),3.500000
'Salem's Lot (2004),5.000000
'Til There Was You (1997),4.000000
'Tis the Season for Love (2015),1.500000
"'burbs, The (1989)",3.176471
'night Mother (1986),3.000000
(500) Days of Summer (2009),3.666667


In [208]:
ratings.sort_values('no_of_ratings',ascending=False).head(10)

,no_of_ratings
title,
Gena the Crocodile (1969),5.0
True Stories (1986),5.0
Cosmic Scrat-tastrophe (2015),5.0
Love and Pigeons (1985),5.0
Red Sorghum (Hong gao liang) (1987),5.0
"Thin Line Between Love and Hate, A (1996)",5.0
Lesson Faust (1994),5.0
Eva (2011),5.0
Who Killed Chea Vichea? (2010),5.0


In [209]:
movie_ratingCount= (df.
     groupby('title')['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'}))
movie_ratingCount.head(10)

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
5,'Tis the Season for Love (2015),1
6,"'burbs, The (1989)",17
7,'night Mother (1986),1
8,(500) Days of Summer (2009),42
9,*batteries not included (1987),7


In [210]:
movie_with_totalRatingCount= df.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
movie_with_totalRatingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [211]:
popularity_threshold = 50
query_string = f'totalRatingCount >= {popularity_threshold}'
rating_popular_movie = movie_with_totalRatingCount.query(query_string)
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [212]:
## First lets create a Pivot matrix

movie_features_df=rating_popular_movie.pivot_table(index='userId',columns='title',values='rating')
movie_features_df.head()

title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),...,Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,5.0,5.0,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN
4,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
from unittest import removeResult
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
train_data, test_data = train_test_split(df, test_size=0.2)
train_movie_features_df = train_data.pivot_table(index='userId', columns='title', values='rating')


In [214]:
def getRecommendation(title):
    user_rating= movie_features_df[title]
    similar_to_movie= movie_features_df.corrwith(user_rating)
    corr_book= pd.DataFrame(similar_to_movie,columns=['Correlation'])
    corr_book.dropna(inplace=True)
    corr_book1= corr_book.sort_values('Correlation',ascending=False).head(10)
    user_ratings_test = test_data[test_data['userId'].isin(train_movie_features_df.index)]
    user_ratings_test = user_ratings_test[['userId', 'movieId', 'rating']]
    user_ratings_test = user_ratings_test.rename(columns={'rating': 'actual_rating'})
    
    user_ratings_test['predicted_rating'] = user_ratings_test['movieId'].apply(lambda x: train_movie_features_df[title][x] if x in train_movie_features_df.columns else 0)
    
    rmse = math.sqrt(mean_squared_error(user_ratings_test['actual_rating'], user_ratings_test['predicted_rating']))
    mae = mean_absolute_error(user_ratings_test['actual_rating'], user_ratings_test['predicted_rating'])

    print("RMSE:", rmse)
    print("MAE:", mae)
    return corr_book1


In [215]:
results=getRecommendation('X-Men: The Last Stand (2006)')
results

RMSE: 3.661078818576166
MAE: 3.5115281634272115


,Correlation
title,
X-Men: The Last Stand (2006),1.000000
Dead Man Walking (1995),0.790724
Johnny Mnemonic (1995),0.783349
"Game, The (1997)",0.747479
Gangs of New York (2002),0.690433
"Lock, Stock & Two Smoking Barrels (1998)",0.684085
Minority Report (2002),0.684016
Grumpier Old Men (1995),0.664586
"Blair Witch Project, The (1999)",0.661438


Matrix Factorization

In [216]:
from sklearn.decomposition import NMF

# Load the user-item rating data (replace 'ratings.csv' with your dataset)
ratings_df = pd.read_csv('ratings.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(ratings_df, test_size=0.2, random_state=42)

# Create a user-item matrix (pivot table)
user_item_matrix = train_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Convert the user-item matrix to a numpy array
user_item_matrix_np = user_item_matrix.to_numpy()

# Define the number of latent factors for NMF
n_latent_factors = 10  # Define the number of latent factors

# Perform Non-Negative Matrix Factorization (NMF)
model = NMF(n_components=n_latent_factors, init='random', random_state=42)
W = model.fit_transform(user_item_matrix_np)
H = model.components_

# Make predictions for the test set
predicted_ratings = np.dot(W, H)

print(predicted_ratings)

# Extract user and movie IDs from the test data
user_ids = test_data['userId'].values
movie_ids = test_data['movieId'].values

# Filter the test data to include only valid user and movie IDs
valid_indices = np.where((user_ids < user_item_matrix.shape[0]) & (movie_ids < user_item_matrix.shape[1]))
user_ids = user_ids[valid_indices]
movie_ids = movie_ids[valid_indices]

# Calculate RMSE only for valid user-item pairs
valid_test_ratings = test_data['rating'].values[valid_indices]
valid_predicted_ratings = predicted_ratings[user_ids, movie_ids]
rmse = np.sqrt(mean_squared_error(valid_test_ratings, valid_predicted_ratings))

print(f'RMSE: {rmse:.2f}')


RMSE: 3.59
